In [1]:
# Basic packages
import pandas as pd 
import numpy as np
import re
import collections
# import matplotlib.pyplot as plt
from pathlib import Path

import pickle

# Packages for data preparation
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

# Packages for modeling
from keras import models
from keras import layers
from keras import regularizers


Using TensorFlow backend.


In [2]:
NB_WORDS = 10000  # Parameter indicating the number of words we'll put in the dictionary
VAL_SIZE = 1000  # Size of the validation set
NB_START_EPOCHS = 20  # Number of epochs we usually start to train with
BATCH_SIZE = 512  # Size of the batches used in the mini-batch gradient descent
MAX_LEN = 24  # Maximum number of words in a sequence
GLOVE_DIM = 50  # Number of dimensions of the GloVe word embeddings
INPUT_PATH = '../input'  # Path where all input files are stored


In [3]:
root = Path('./')
input_path = root / 'input/' 
ouput_path = root / 'output/'
source_path = root / 'source/'


In [4]:

def deep_model(model, X_train, y_train, X_valid, y_valid):
    '''
    Function to train a multi-class model. The number of epochs and 
    batch_size are set by the constants at the top of the
    notebook. 
    
    Parameters:
        model : model with the chosen architecture
        X_train : training features
        y_train : training target
        X_valid : validation features
        Y_valid : validation target
    Output:
        model training history
    '''
    model.compile(optimizer='rmsprop'
                  , loss='categorical_crossentropy'
                  , metrics=['accuracy'])
    
    model.fit(X_train
                       , y_train
                       , epochs=NB_START_EPOCHS
                       , batch_size=BATCH_SIZE
                       , validation_data=(X_valid, y_valid)
                       , verbose=1)
    model.save("./output/model/model.h5")
    

In [5]:
def eval_metric(history, metric_name):
    '''
    Function to evaluate a trained model on a chosen metric. 
    Training and validation metric are plotted in a
    line chart for each epoch.
    
    Parameters:
        history : model training history
        metric_name : loss or accuracy
    Output:
        line chart with epochs of x-axis and metric on
        y-axis
    '''
    metric = history.history[metric_name]
    val_metric = history.history['val_' + metric_name]

    e = range(1, NB_START_EPOCHS + 1)

    plt.plot(e, metric, 'bo', label='Train ' + metric_name)
    plt.plot(e, val_metric, 'b', label='Validation ' + metric_name)
    plt.legend()
    plt.show()


In [6]:
def test_model(model, X_train, y_train, X_test, y_test, epoch_stop):
    '''
    Function to test the model on new data after training it
    on the full training data with the optimal number of epochs.
    
    Parameters:
        model : trained model
        X_train : training features
        y_train : training target
        X_test : test features
        y_test : test target
        epochs : optimal number of epochs
    Output:
        test accuracy and test loss
    '''
    model.fit(X_train
              , y_train
              , epochs=epoch_stop
              , batch_size=BATCH_SIZE
              , verbose=0)
    results = model.evaluate(X_test, y_test)
    
    return results

In [7]:
def remove_stopwords(input_text):
    '''
    Function to remove English stopwords from a Pandas Series.
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    stopwords_list = stopwords.words('english')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words) 

In [8]:

def remove_mentions(input_text):
    '''
    Function to remove mentions, preceded by @, in a Pandas Series
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    return re.sub(r'@\w+', '', input_text)


In [9]:

df = pd.read_csv(input_path / 'train.csv')
df = df.reindex(np.random.permutation(df.index))  
df = df[['comment_text', 'toxic']]
df.text = df.comment_text.apply(remove_stopwords).apply(remove_mentions)


/home/dell-3060/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.comment_text, df.toxic, test_size=0.1, random_state=37)
print('# Train data samples:', X_train.shape[0])
print('# Test data samples:', X_test.shape[0])
assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]


# Train data samples: 143613
# Test data samples: 15957


In [11]:
tk = Tokenizer(num_words=NB_WORDS,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ")
tk.fit_on_texts(X_train)
# print(tk)
# saving
with open('./output/model/tk.pickle', 'wb') as handle:
    pickle.dump(tk, handle, protocol=pickle.HIGHEST_PROTOCOL)
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)



In [12]:

seq_lengths = X_train.apply(lambda x: len(x.split(' ')))
seq_lengths.describe()

count    143613.000000
mean         67.995648
std         100.820994
min           1.000000
25%          17.000000
50%          36.000000
75%          76.000000
max        2273.000000
Name: comment_text, dtype: float64

In [13]:
X_train_seq_trunc = pad_sequences(X_train_seq, maxlen=MAX_LEN)
X_test_seq_trunc = pad_sequences(X_test_seq, maxlen=MAX_LEN)


In [14]:
X_train_seq_trunc[10]  # Example of padded sequence

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   9,  44,  16, 283,  94], dtype=int32)

In [15]:
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)



In [16]:
X_train_emb, X_valid_emb, y_train_emb, y_valid_emb = train_test_split(X_train_seq_trunc, y_train_oh, test_size=0.1, random_state=37)

assert X_valid_emb.shape[0] == y_valid_emb.shape[0]
assert X_train_emb.shape[0] == y_train_emb.shape[0]

print('Shape of validation set:',X_valid_emb.shape)


Shape of validation set: (14362, 24)


In [17]:
glove_file = 'glove.twitter.27B.25d.txt'
glove_dir = 'glove/'
emb_dict = {}
glove = open('input/glove/glove.twitter.27B.25d.txt')
for line in glove:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    emb_dict[word] = vector
glove.close()


In [18]:
print("*")
profanity_words = ['fuck', 'pussy', 'sad', 'hell']
for w in profanity_words:
    if w in emb_dict.keys():
        print('Found the word {} in the dictionary'.format(w))



*
Found the word fuck in the dictionary
Found the word pussy in the dictionary
Found the word sad in the dictionary
Found the word hell in the dictionary


In [19]:
GLOVE_DIM = 25
emb_matrix = np.zeros((NB_WORDS, GLOVE_DIM))

for w, i in tk.word_index.items():
    # The word_index contains a token for all words of the training data so we need to limit that
    if i < NB_WORDS:
        vect = emb_dict.get(w)
        # Check if the word from the training data occurs in the GloVe word embeddings
        # Otherwise the vector is kept with only zeros
        if vect is not None:
            emb_matrix[i] = vect
    else:
        break

In [20]:
from keras.layers import LSTM 
lstm_out = 20

emb_model2 = models.Sequential()
emb_model2.add(layers.Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN))
emb_model2.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
emb_model2.add(layers.Dense(2, activation='softmax'))
emb_model2.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 24, 25)            250000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                3680      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 42        
Total params: 253,722
Trainable params: 253,722
Non-trainable params: 0
_________________________________________________________________


In [21]:
emb_history2 = deep_model(emb_model2, X_train_emb, y_train_emb, X_valid_emb, y_valid_emb)

Instructions for updating:
Use tf.cast instead.
Train on 129251 samples, validate on 14362 samples
Epoch 1/20
129251/129251 [==============================] - 11s 86us/step - loss: 0.2266 - acc: 0.9258 - val_loss: 0.1419 - val_acc: 0.9506
Epoch 2/20
129251/129251 [==============================] - 5s 42us/step - loss: 0.1401 - acc: 0.9532 - val_loss: 0.1271 - val_acc: 0.9572
Epoch 3/20
129251/129251 [==============================] - 5s 42us/step - loss: 0.1305 - acc: 0.9560 - val_loss: 0.1241 - val_acc: 0.9603
Epoch 4/20
129251/129251 [==============================] - 5s 42us/step - loss: 0.1270 - acc: 0.9574 - val_loss: 0.1223 - val_acc: 0.9603
Epoch 5/20
129251/129251 [==============================] - 6s 43us/step - loss: 0.1244 - acc: 0.9577 - val_loss: 0.1214 - val_acc: 0.9606
Epoch 6/20
129251/129251 [==============================] - 6s 43us/step - loss: 0.1230 - acc: 0.9582 - val_loss: 0.1211 - val_acc: 0.9611
Epoch 7/20
129251/129251 [==============================] - 5s 41u

In [22]:
emb_results2 = test_model(emb_model2, X_train_seq_trunc, y_train_oh, X_test_seq_trunc, y_test_oh, 3)
print('/n')
print('Test accuracy of word embedding model 2: {0:.2f}%'.format(emb_results2[1]*100))

15957/15957 [==============================] - 1s 53us/step
/n
Test accuracy of word embedding model 2: 95.53%


In [68]:
twt = ["fuck"]
print(type(twt))
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tk.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=24, dtype='int32', value=0)
print(twt)
sentiment = emb_model2.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("positive")
elif (np.argmax(sentiment) == 1):
    print("negative")




<class 'list'>
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0 128]]
negative


In [69]:
model = load_model('output/model/model.h5')
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print("senti:",sentiment)
if(np.argmax(sentiment) == 0):
    print("positive")
elif (np.argmax(sentiment) == 1):
    print("negative")



senti: [0.00512103 0.994879  ]
negative


In [70]:
def list_string(strn):
  level=[]
  def permute(prefix, suffix):
    level.append(prefix)
    if len(suffix)==0:
       return
    for i in range(len(suffix)):
      permute(prefix + suffix[i], suffix[:i]+suffix[i+1:])
  permute("",strn)
  return level

In [74]:
twt = list_string('fuckboy')
print(type(a))
if "fuck" in twt:
    print("true")

<class 'str'>
true


In [80]:
result = []
for w in twt:
#     print([w])
    b = [w]
#     print(b)
    w = tk.texts_to_sequences(b)
#     padding the tweet to have exactly the same shape as `embedding_2` input
    b = pad_sequences(b, maxlen=24, dtype='int32', value=0)
    sentiment = model.predict(b,batch_size=1,verbose = 2)[0]
    if(np.argmax(sentiment) == 0):
        a = "positive"
    elif (np.argmax(sentiment) == 1):
        a = "negative"
    result.append(a)
count = 0
if 'negative' in  result:
    count += 1

print(count)
    

ValueError: invalid literal for int() with base 10: 'f'

In [65]:
result

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',